In [1]:
di = '../data/unzipped'

In [2]:
from subprocess import run
run('make clean; make run',shell=True) # when run excludes extract() call

rm -rf __pycache__
rm -f ../data/unzipped/*
rm -f *.pyc
rm -f .DS_Store
rm -f ../.DS_Store
rm -f ../data/.DS_Store
venv/bin/python3 clean.py 


/Users/hudsonnash/Dropbox/Mac/Desktop/Su22s/phil/phil/control/venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


./convert.sh ../data/unzipped


CompletedProcess(args='make clean; make run', returncode=0)

In [3]:
import os
os.listdir(di)

['10-ManilaCity2021_Part3-Status_of_PYs_Recomm.docx',
 '02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.doc',
 '02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.pdf',
 '10-ManilaCity2021_Part3-Status_of_PYs_Recomm.pdf']

In [4]:
from extracttools import *
di = '../data/unzipped'

NameError: name 'dfs' is not defined

In [7]:
from fuzzywuzzy import fuzz
import config as conf
def add_date_cols(df,fn): # add file download date from metadata and other dates from metadata and date or year published
    pass # TODO: 11/30/22
def add_location_col(df,fn): # add location from filename
    pass # TODO: 11/30/22
def good_match(og:str,ref:list[str]):
    good = ''
    approx = 0
    og = og.lower()
    for target in ref:
        new_approx = fuzz.token_sort_ratio(og,target)
        if new_approx > approx:
            approx = new_approx
            good = target
    if approx < 60:
        good = og
    return good
def polish(df):
    # remove \r characters
    df = df.rename(lambda s:s.replace('\r',' '),axis='columns') # remove '\r' from headers
    df = df.replace('\r','',regex=True) # replace all '\r' throughout
    # approximate headers
    cols = df.columns
    newcols = [good_match(col,conf.CANON_HEADERS) for col in cols]
    df.columns = newcols
    return df
def overflow_check(df,ref,main_df):
    head = df.head(10) # Can set this to be the necessary number of rows -- typically overflow only lasts for 2-3 rows
    headers_at = 0
    for row in range(len(head)):
        if sum([bool(any([fuzz.token_sort_ratio(cell,target) > 60 for target in ref])) for cell in head.loc[row]]) < 2: # if there are less than 2 near-canonical headers
            if sum([bool(any([fuzz.token_sort_ratio(cell,target) > 60 for target in ref])) for cell in head.loc[row]]) > 4: # if there are more than 4 near-canonical headers in the row
                headers_at = row
                overflow = df.columns.values
                df = polish(df)
                overflow = pd.DataFrame(overflow,columns = ref)
                main_df = main_df.append(overflow,ignore_index=True)
                df = df.loc[:headers_at+1]
            df = polish(df)
            df.loc[headers_at,'audit observation'] = '!PHIL EXCEPTION: Dataframe columns are faulty and data has been lost. Refer to the document to find the source of error and report the bug in the repository: https://github.com/hudnash/phil/issues'
def extract(di):
    target_page = -1
    table_end_page = -1
    out = pd.DataFrame()
    file_counter = 1
    for filename in os.listdir(di):
        print('Looking for file n = '+str(file_counter)+'...')
        no_df_found = True
        if os.path.splitext(filename)[1] == ".pdf":
            filename = os.path.join(di,filename)
            print('Now scanning '+filename)
            pdf = p.PdfFileReader(filename)
            page_count = pdf.numPages
            for pagenum in range(page_count):
                print('Looking for Part 3 on p. '+str(pagenum+1)+' of '+str(page_count)+' pages')
                page = pdf.getPage(pagenum)
                page_content = page.extractText()
                if all([target in page_content.lower() for target in conf.TARGET_SENTENCE]):
                    no_df_found = False
                    target_page = pagenum
                    delete_pages_before(filename,target_page)
                    dfs = default(filename) # feed chunks of PDF table scraped via tabula/io.py
                    if len(dfs) > 1:
                        count = 1
                        for df in dfs:
                            print(str(count)+' of '+str(len(dfs))+' dataframes collected.')
                            df = polish(df)
                            overflow_check(df,conf.CANON_HEADERS,out)
                            #df = add_filename_col(filename,df) # add a filename column
                            #df = add_location_col(df,filename)
                            #df = add_date_cols(df,filename)
                            out = out.append(df,ignore_index=True) #TODO 11/30/22: Approximate header string matching during append
                            count = count + 1
                    elif len(dfs) == 1:
                        df = add_filename_col(filename,dfs[0])
                        df = polish(df)
                        overflow_check(df,conf.CANON_HEADERS,out)
                        out = out.append(df,ignore_index=True) #TODO 11/30/22: Approximate header string matching during append
                        print('1 of 1 dataframe collected.')
        if no_df_found:
            print('Exception: Could not locate tables in file #'+str(file_counter)+' '+filename+'\nTry: Adjust target sequences list in config.py')
        file_counter = file_counter + 1
    return out

In [8]:
df = extract(di)

Looking for file n = 1...
Exception: Could not locate tables in file #1 10-ManilaCity2021_Part3-Status_of_PYs_Recomm.docx
Try: Adjust target sequences list in config.py
Looking for file n = 2...
Exception: Could not locate tables in file #2 02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.doc
Try: Adjust target sequences list in config.py
Looking for file n = 3...
Now scanning ../data/unzipped/02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.pdf
Looking for Part 3 on p. 1 of 3 pages
1 of 2 dataframes collected.
2 of 2 dataframes collected.
Looking for Part 3 on p. 2 of 3 pages
1 of 1 dataframe collected.
Looking for Part 3 on p. 3 of 3 pages
Looking for file n = 4...
Now scanning ../data/unzipped/10-ManilaCity2021_Part3-Status_of_PYs_Recomm.pdf
Looking for Part 3 on p. 1 of 34 pages
1 of 33 dataframes collected.
2 of 33 dataframes collected.
3 of 33 dataframes collected.
4 of 33 dataframes collected.
5 of 33 dataframes collected.
6 of 33 dataframes collected

In [9]:
df

,audit observation,recommendations,references,management action,status of implementation,reasons for partial/non-implementation,filename,observations and recommendations
0,1. At end of audit period the Accountant ha...,The City Mayor shall require the OIC CityAccou...,Page 13 and 14CY 2009 AAR,NaN,Implemented,NaN,NaN,NaN
1,2.The Accountant failed to submit to the Audit...,We recommend that the City Mayor shall direct ...,Page 14 and 15CY 2009 AAR,NaN,implemented,NaN,NaN,NaN
2,3.The accuracy and validity of the balance of ...,Itisrecommendedthatthelocalofficialconcernedsh...,Page 15 and 16CY 2009 AAR,NaN,UnimplementedReiterated inPart II,NaN,NaN,NaN
3,"4Purchase Orders, contracts and reports on th...",It is recommended that the concerned agency of...,Page 16-18CY 2009 AAR,NaN,Unimplemented,NaN,NaN,NaN
4,"5.The LGU appropriated a total amount ofP1,896...",We recommend that 5% of the total IRA ...,Page 18 and19CY 2009 AAR,NaN,Unimplemented,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
114,NaN,NaN,AAR2017Observation No. 31Pages133-138,NaN,NaN,NaN,NaN,28.The ultimate goals andobjectives of the C...
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,We recommended thatManagement:
116,NaN,NaN,NaN,Lack of effectiveactionbytheconcernedofficials.,Not Implemented,Modification andrepairoftheCTISMhasnotbeencoo...,NaN,a.RequiretheconcernedCityofficialstocoordinate...
117,NaN,NaN,NaN,Lack of effectiveactionbytheconcernedofficials.,Not Implemented,Extensive trainingforthehospitalpersonnel has...,NaN,b.Makeproperrepresentationwiththe contractor f...


In [ ]:
head = df.head(5)
head

In [ ]:
head.loc[0]

In [ ]:
head.columns

In [ ]:
len(head)